In [1]:
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import pandas as pd
import re
import datetime
import calendar
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Find all Fridays in March from the years 2014-2019

date_list = []
for year in range(2015,2020):
    cal = calendar.monthcalendar(year,3)
    year = str(year)
    year = year[2:]
    for cal_week in cal:
        if cal_week[5] !=0:
            date_list.append(str(year)+'03'+str(cal_week[5]))
           
for i in range(len(date_list)):
    if len(date_list[i]) == 5:
        date_with_0 = date_list[i][:4]+'0'+date_list[i][4]
        date_list.append(date_with_0)
date_list

SyntaxError: invalid syntax (<ipython-input-2-e39d0b9dfb8e>, line 1)

In [3]:
#Scrape and concatenate data
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt" #url with formatting to fill in the number of the particular week
    dfs = []
    for week_num in week_nums:
        #print(week_num)#for each week
        if len(week_num) == 6:
            file_url = url.format(week_num) #format the url so it grabs that week
            dfs.append(pd.read_csv(file_url))#append to our empty list
        else:
            pass
    return pd.concat(dfs) #combine files for each week
#week_nums = re.compile('..03..') it would be cool to just use a regex here
turnstiles_df = get_data(date_list)
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns] #use our function to get a df with all of our data


In [4]:
#Put data into the mta.db
engine = create_engine("sqlite:///mta.db")
turnstiles_df.to_sql('Project_data', con = engine, if_exists = 'replace')
turnstiles_df = pd.read_sql_table('Project_data', con = engine)
turnstiles_df.to_pickle('turnstiles_df.pickle')


In [5]:
# turnstiles_df = pd.read_pickle('turnstiles_df.pickle')
#Create date_time columns as datetime object
turnstiles_df['DATE_TIME'] = turnstiles_df['DATE'] + " " + turnstiles_df['TIME']
turnstiles_df['DATE_TIME'] = pd.to_datetime(turnstiles_df['DATE_TIME'])

#Change station names to be uniform
mymap = {'AV$':'AVE', 'STS$':'ST', '/':' ', '-':' ', 'CO$': 'COL', 'PLACE$':'PL', 'AVENUE$':'AVE', 'PARKWAY$': 'PWY', 'PK$':'PARK', 'BL$':'BLVD', 'ROAD$':'RD'}
for abbr1, abbr2 in mymap.items():
    turnstiles_df['STATION'] = turnstiles_df['STATION'].str.replace(abbr1, abbr2, regex=True)
all_ts = turnstiles_df.STATION.unique()
turnstiles_df.to_pickle('turnstiles_df2.pickle')

In [6]:
#Get rid of non-regular and duplicate entries
#turnstiles_df = pd.read_pickle('turnstiles_df2.pickle')
turnstiles_df['YEAR'] = pd.DatetimeIndex(turnstiles_df['DATE_TIME']).year
turnstiles_df.sort_values(by = ["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, ascending = True)
turnstiles_df = turnstiles_df[(turnstiles_df['DESC'] == 'REGULAR')].copy()
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)
turnstiles_df.to_pickle('turnstiles_df3.pickle')


In [7]:
#find the total exits and entries from cumulative
# turnstiles_df = pd.read_pickle('turnstiles_df3.pickle')
turnstiles_df[['TOTAL ENTRIES', 'TOTAL EXITS']] = (turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION", "YEAR"])[["ENTRIES", 'EXITS']].diff(1)
turnstiles_df.to_pickle('turnstiles_df4.pickle')


In [24]:
#Remove Na values from total entries and total exits 
turnstiles_df = pd.read_pickle('turnstiles_df4.pickle')
turnstiles_df.fillna(0)

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,YEAR,TOTAL ENTRIES,TOTAL EXITS
3330795,0,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,03:00:00,REGULAR,5562172,1877574,2016-02-27 03:00:00,2016,0.0,0.0
3330796,1,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,07:00:00,REGULAR,5562184,1877586,2016-02-27 07:00:00,2016,12.0,12.0
3330797,2,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,11:00:00,REGULAR,5562267,1877691,2016-02-27 11:00:00,2016,83.0,105.0
3330798,3,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,15:00:00,REGULAR,5562516,1877745,2016-02-27 15:00:00,2016,249.0,54.0
3330799,4,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,19:00:00,REGULAR,5562920,1877821,2016-02-27 19:00:00,2016,404.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139203,204590,TRAM2,R469,00-05-01,RIT ROOSEVELT,R,RIT,03/29/2019,05:00:00,REGULAR,5554,370,2019-03-29 05:00:00,2019,0.0,0.0
3139204,204591,TRAM2,R469,00-05-01,RIT ROOSEVELT,R,RIT,03/29/2019,09:00:00,REGULAR,5554,374,2019-03-29 09:00:00,2019,0.0,4.0
3139205,204592,TRAM2,R469,00-05-01,RIT ROOSEVELT,R,RIT,03/29/2019,13:00:00,REGULAR,5554,374,2019-03-29 13:00:00,2019,0.0,0.0
3139206,204593,TRAM2,R469,00-05-01,RIT ROOSEVELT,R,RIT,03/29/2019,17:00:00,REGULAR,5554,374,2019-03-29 17:00:00,2019,0.0,0.0


In [25]:
#Do negative counters for entries and exits exist at same turnstiles only? No
negcount_exitandentry = turnstiles_df[(turnstiles_df["TOTAL ENTRIES"] < 0) & (turnstiles_df['TOTAL EXITS'] < 0)]
negcount_exitorentry = turnstiles_df[(turnstiles_df["TOTAL ENTRIES"] < 0) | (turnstiles_df['TOTAL EXITS'] < 0)]
print(negcount_exitandentry.shape)
print(negcount_exitorentry.shape)

(14921, 16)
(43334, 16)


In [28]:
#Get absolute values of total entries and total exits with negative values 
turnstiles_df['TOTAL ENTRIES POS'] = turnstiles_df['TOTAL ENTRIES'].abs()
turnstiles_df['TOTAL EXITS POS'] = turnstiles_df['TOTAL EXITS'].abs()
turnstiles_df.to_pickle('/Users/emilylorenzen/NBM_EDA_Student/Metis_EDA/turnstile_df5.pickle')

In [3]:
#Remove outliers, currently not the correct method - should use outliers in Poisson distribution
turnstiles_df = pd.read_pickle('/Users/emilylorenzen/NBM_EDA_Student/Metis_EDA/turnstile_df5.pickle')
turnstiles_df = turnstiles_df[turnstiles_df['TOTAL EXITS POS'] < turnstiles_df['TOTAL EXITS POS'].quantile(.997)].copy()
turnstiles_df = turnstiles_df[turnstiles_df['TOTAL ENTRIES POS'] < turnstiles_df['TOTAL ENTRIES POS'].quantile(.997)].copy()
# turnstiles_df.to_pickle('turnstiles_df6.pickle')


In [4]:
#DROP SOME EXTRANEOUS COLUMNS, RENAME COLUMNS TO MAKE SENSE AND ADD A FEW MORE COLUMNS
turnstiles_df_clean = turnstiles_df.drop(['index', 'DESC', 'ENTRIES', 'EXITS', 'TOTAL ENTRIES', 'TOTAL EXITS'], axis = 1)
turnstiles_df_clean.rename(columns = {'TOTAL ENTRIES POS': 'ENTRIES', 'TOTAL EXITS POS': 'EXITS'}, inplace = True)
turnstiles_df_clean['BUSYNESS'] = turnstiles_df_clean['EXITS'] + turnstiles_df_clean['ENTRIES']
turnstiles_df_clean['DAY_OF_WEEK_NUM'] = turnstiles_df_clean['DATE_TIME'].dt.dayofweek


In [5]:
#READ DATAFRAME INTO SQL 
from sqlalchemy import create_engine
engine = create_engine("sqlite:///mta.db")
turnstiles_df_clean.to_sql('turnstiles_df_clean', con = engine)

In [23]:
#IGNORE THIS - CODE TO WRITE FOR PROPER OUTLIER FILTERING LATER

#from scipy.stats import (kstest, poisson)
#GET RID OF OUTLITERS 
#First determine if the data is Gaussian to decide which method to use

def is_gaussian(data):
    from scipy.stats import normaltest
    stat, p = normaltest(data.to_numpy())  
    alpha = 0.05
    if p > alpha:
	    print('Sample looks Gaussian (fail to reject H0)')
    else:
	    print('Sample does not look Gaussian (reject H0)')

is_gaussian(turnstiles_df['TOTAL ENTRIES'])
is_gaussian(turnstiles_df['TOTAL EXITS'])

def find_outliers_nonnorm(data):
    


Sample does not look Gaussian (reject H0)
Sample does not look Gaussian (reject H0)
